## Imports

In [45]:
#Tratamiento de datos
import pandas as pd
import numpy as np

#Visualización de datos
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import plot_importance
#from matplotlib import rcParams

#Análisis, modelado, predicción y métricas
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBRegressor
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
import statsmodels.tsa.api as smt

import pickle
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

#import statsmodels.api as smz
#import statsmodels.formula.api as smf
#from sklearn.metrics import r2_score
#from statsmodels.tsa.holtwinters import SimpleExpSmoothing
#from sklearn.model_selection import TimeSeriesSplit 
#from sklearn.metrics import mean_squared_error
#from sklearn.model_selection import GridSearchCV
#from sklearn.model_selection import StratifiedKFold
#import lightgbm as lgb
#from statsmodels.tsa.stattools import acf, pacf
#from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
#import statsmodels.formula.api as smf


#Quitar warnings
import warnings
warnings.filterwarnings('ignore')

In [46]:
#pip install xgboost

In [47]:
#!pip install openpyxl==3.0.10

## Data preparation

In [48]:
#Leemos el archivo y miramos las primeras cinco filas del dataset
data = pd.read_excel('../TP4/datos.xlsx', engine='openpyxl')

In [49]:
# en el caso que cargue los NaN ejecutar el dropna
data = data.dropna()

In [50]:
#Armamos una lista con los nuevos nombres de las columnas
columnas = ["año", "mes", "numero_mes", "estacion", "semana", "fecha", "tipo_dia", "dia", \
            "numero_dia", "energia_sadi", "potencia_pico", "hora_pico", "temp", "estado_tiempo" ]

#Asignamos el nuevo nombre
data.columns = columnas

In [51]:
data['nueva_estacion'] = data['estacion']

In [52]:
#creación de estaciones PRIMAVERA y OTOÑO
mask_otoño = ((data['fecha'] > '2007-03-20 00:00:00') & (data['fecha'] < '2007-06-22 00:00:00') | \
              (data['fecha'] > '2008-03-20 00:00:00') & (data['fecha'] < '2008-06-22 00:00:00') | \
              (data['fecha'] > '2009-03-20 00:00:00') & (data['fecha'] < '2009-06-22 00:00:00') | \
              (data['fecha'] > '2010-03-20 00:00:00') & (data['fecha'] < '2010-06-22 00:00:00') | \
              (data['fecha'] > '2011-03-20 00:00:00') & (data['fecha'] < '2011-06-22 00:00:00') | \
              (data['fecha'] > '2012-03-20 00:00:00') & (data['fecha'] < '2012-06-22 00:00:00') | \
              (data['fecha'] > '2013-03-20 00:00:00') & (data['fecha'] < '2013-06-22 00:00:00') | \
              (data['fecha'] > '2014-03-20 00:00:00') & (data['fecha'] < '2014-06-22 00:00:00') | \
              (data['fecha'] > '2015-03-20 00:00:00') & (data['fecha'] < '2015-06-22 00:00:00') | \
              (data['fecha'] > '2016-03-20 00:00:00') & (data['fecha'] < '2016-06-22 00:00:00') | \
              (data['fecha'] > '2017-03-20 00:00:00') & (data['fecha'] < '2017-06-22 00:00:00') | \
              (data['fecha'] > '2018-03-20 00:00:00') & (data['fecha'] < '2018-06-22 00:00:00') | \
              (data['fecha'] > '2019-03-20 00:00:00') & (data['fecha'] < '2019-06-22 00:00:00') | \
              (data['fecha'] > '2020-03-20 00:00:00') & (data['fecha'] < '2020-06-22 00:00:00') | \
              (data['fecha'] > '2021-03-20 00:00:00') & (data['fecha'] < '2021-06-22 00:00:00') | \
              (data['fecha'] > '2022-03-20 00:00:00') & (data['fecha'] < '2022-06-22 00:00:00'))

data.loc[mask_otoño,'nueva_estacion'] = 'OTOÑO'

In [53]:
mask_primavera = ((data['fecha'] > '2007-09-21 00:00:00') & (data['fecha'] < '2007-12-22 00:00:00') | \
                 (data['fecha'] > '2008-09-21 00:00:00') & (data['fecha'] < '2008-12-22 00:00:00') | \
                 (data['fecha'] > '2009-09-21 00:00:00') & (data['fecha'] < '2009-12-22 00:00:00') | \
                 (data['fecha'] > '2010-09-21 00:00:00') & (data['fecha'] < '2010-12-22 00:00:00') | \
                 (data['fecha'] > '2011-09-21 00:00:00') & (data['fecha'] < '2011-12-22 00:00:00') | \
                 (data['fecha'] > '2012-09-21 00:00:00') & (data['fecha'] < '2012-12-22 00:00:00') | \
                 (data['fecha'] > '2013-09-21 00:00:00') & (data['fecha'] < '2013-12-22 00:00:00') | \
                 (data['fecha'] > '2014-09-21 00:00:00') & (data['fecha'] < '2014-12-22 00:00:00') | \
                 (data['fecha'] > '2015-09-21 00:00:00') & (data['fecha'] < '2015-12-22 00:00:00') | \
                 (data['fecha'] > '2016-09-21 00:00:00') & (data['fecha'] < '2016-12-22 00:00:00') | \
                 (data['fecha'] > '2017-09-21 00:00:00') & (data['fecha'] < '2017-12-22 00:00:00') | \
                 (data['fecha'] > '2018-09-21 00:00:00') & (data['fecha'] < '2018-12-22 00:00:00') | \
                 (data['fecha'] > '2019-09-21 00:00:00') & (data['fecha'] < '2019-12-22 00:00:00') | \
                 (data['fecha'] > '2020-09-21 00:00:00') & (data['fecha'] < '2020-12-22 00:00:00') | \
                 (data['fecha'] > '2021-09-21 00:00:00') & (data['fecha'] < '2021-12-22 00:00:00') | \
                 (data['fecha'] > '2022-09-21 00:00:00') & (data['fecha'] < '2022-12-22 00:00:00'))

data.loc[mask_primavera,'nueva_estacion'] = 'PRIMAVERA'

In [54]:
#paso toda la columna a minúsculas para mayor comodidad de trabajo
data['nueva_estacion'] = data['nueva_estacion'].str.lower()

In [55]:
#Cambiamos el índice del Dataframe usando el método PeriodIndex de pandas y ponemos frecuencia diaria.
data.index=pd.PeriodIndex(data.fecha,  freq='D')

In [56]:
#Generamos la variable 'timeindex' como variable dummy de tiempo
data['timeindex'] = pd.Series(np.arange(len(data.energia_sadi)), index=data.index)

In [57]:
#Vamos a convertir los meses en dummies con el método get_dummies
#Aplicamos drop_first= True para envitar la colinealidad entre variables.

#las siguientes dos fueron agregadas porque en linux no me toma el integer de una, sino que las toma como float
#comentar en el caso que no sean necesarias
data['numero_mes'] = data['numero_mes'].astype(int)
data['numero_mes'] = data['numero_mes'].astype(str)


data_dummie_mes =pd.get_dummies(data['numero_mes'], prefix='mes', drop_first=True)   


#Vamos a convertir el resto de las variables categóricas en dummies.
data_dummies=pd.get_dummies(data[['tipo_dia', 'dia', 'estado_tiempo', 'nueva_estacion']], drop_first=True )

#Unimos los dos dataframes de dummies y el que teniamos originalmente
data_dm=data_dummie_mes.join(data_dummies)
data=data.join(data_dm)

In [58]:
data.rename(columns = {'fecha':'date'}, inplace = True)

In [59]:
#Separamos en train y test para que solo tome las features necesarias.

x_train = data.drop(['año', 'mes', 'numero_mes', 'estacion', 'semana', 'date', 'tipo_dia',
       'dia', 'numero_dia', 'energia_sadi', 'potencia_pico', 'hora_pico','estado_tiempo', 'nueva_estacion','tipo_dia_FERIADO','dia_Jueves', 'dia_Lunes', 'dia_Martes',
       'dia_Miércoles', 'dia_Sábado', 'dia_Viernes'], axis=1)
y_train=data.energia_sadi

In [60]:
nuevas_col = ['temp', 'timeindex', 'mes_10', 'mes_11', 'mes_12', 'mes_2', 'mes_3', \
       'mes_4', 'mes_5', 'mes_6', 'mes_7', 'mes_8', 'mes_9', 'tipo_dia_habil', \
       'tipo_dia_sabado', 'estado_tiempo_N', 'estado_tiempo_SN', \
       'nueva_estacion_otono', 'nueva_estacion_primavera',\
       'nueva_estacion_verano']

In [61]:
x_train.columns = nuevas_col

In [62]:
x_train.columns

Index(['temp', 'timeindex', 'mes_10', 'mes_11', 'mes_12', 'mes_2', 'mes_3',
       'mes_4', 'mes_5', 'mes_6', 'mes_7', 'mes_8', 'mes_9', 'tipo_dia_habil',
       'tipo_dia_sabado', 'estado_tiempo_N', 'estado_tiempo_SN',
       'nueva_estacion_otono', 'nueva_estacion_primavera',
       'nueva_estacion_verano'],
      dtype='object')

## Data Modeling

In [63]:
#Instanciamos el modelo
xgb_regressor = XGBRegressor()

In [64]:
#Entrenamos el modelo
xgb_regressor = xgb_regressor.fit(x_train,y_train)

In [65]:
#Hacemos las predicciones
y_predict = xgb_regressor.predict(x_train)

In [66]:
#Obtengo los residuos entre el dato real y el dato del modelo XGBoost
residuos = y_train - y_predict

In [67]:
model_ARIMA = ARIMA(residuos, order=(1,0,1))

# Estimo el modelo:
results_ARIMA = model_ARIMA.fit()
results_ARIMA.fittedvalues.head()

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           12

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.36392D+00    |proj g|=  2.38476D-05

At iterate    5    f=  3.36392D+00    |proj g|=  7.32747D-06

At iterate   10    f=  3.36392D+00    |proj g|=  1.33227D-07

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    3     12     16      1     0     0   0.000D+00   3.364D+00
  F =   3.3639240395605956     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            


 This problem is unconstrained.


fecha
2007-01-01    0.000838
2007-01-02    0.335467
2007-01-03   -0.430207
2007-01-04   -0.314191
2007-01-05    0.337826
Freq: D, dtype: float64

In [68]:
res_ARIMA =  results_ARIMA.fittedvalues - residuos

In [69]:
predictions_ARIMA, se, conf = results_ARIMA.forecast(len(y_train), alpha=0.05)

In [70]:
xgboost_arima = y_predict + results_ARIMA.fittedvalues

In [71]:
def RMSE(predichos, observados):
    mse = (predichos - observados) ** 2
    rmse = np.sqrt(mse.sum() / mse.count())
    return rmse

In [72]:
#Calculamos el RMSE en Train con ARIMA
RMSE(y_train, y_predict)

7.178049046598337

In [73]:
pipe = Pipeline([('xgboost', XGBRegressor())])

In [74]:
pipe.fit(x_train, y_train)

Pipeline(steps=[('xgboost',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, enable_categorical=False,
                              gamma=0, gpu_id=-1, importance_type=None,
                              interaction_constraints='',
                              learning_rate=0.300000012, max_delta_step=0,
                              max_depth=6, min_child_weight=1, missing=nan,
                              monotone_constraints='()', n_estimators=100,
                              n_jobs=12, num_parallel_tree=1, predictor='auto',
                              random_state=0, reg_alpha=0, reg_lambda=1,
                              scale_pos_weight=1, subsample=1,
                              tree_method='exact', validate_parameters=1,
                              verbosity=None))])

In [76]:
with open ("modelo_entrenado.pkl", "wb") as model:
    pickle.dump(pipe, model)

In [77]:
prueba = pd.DataFrame({'temp': [19.8], 'timeindex': [4685], 'mes_10': [1], 'mes_11': [0], 'mes_12': [0], \
              'mes_2': [0], 'mes_3': [0], 'mes_4': [0], 'mes_5': [0], 'mes_6': [0], 'mes_7': [0], \
              'mes_8': [0], 'mes_9': [0], 'tipo_dia_HÁBIL': [1], 'tipo_dia_SÁBADO': [0], \
              'estado_tiempo_N': [1], 'estado_tiempo_SN': [0], 'nueva_estacion_otoño': [0], \
              'nueva_estacion_primavera': [1], 'nueva_estacion_verano':[0]})

In [78]:
with open('modelo_entrenado.pkl','rb') as f:
    regressor = pickle.load(f)

In [80]:
regressor.predict(prueba)[0]

374.71207